In [1]:
import torch

/lustre/hpc/pheno/inar/mambaforge/envs/torch39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt
from scipy.stats import entropy

import numpy as np
from matplotlib.colors import LogNorm

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [6]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", 
                                          torch_dtype=torch.float16,
                                          device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")


Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.35s/it]


In [7]:
model_gpt = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer_gpt = AutoTokenizer.from_pretrained("gpt2")

In [8]:
from transformers import pipeline, TextStreamer

In [30]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "It's prequel time again. In December, we'll get to "
generated = pipe(prompt, streamer=TextStreamer(tokenizer), max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


It's prequel time again. In December, we'll get to 1984's "The Terminator," and in January, we'll get to 1989's "Terminator 2: Judgment Day."

The first film, which was directed by James Cameron, is about a cyborg assassin from the future who is sent back in time to kill Sarah Connor (Linda Hamilton), the mother of the future leader of the human resistance against the machines. The second film, which was also directed by Cameron, is about the same cyborg assassin, who is sent back in time to protect Sarah's son, John Connor (


In [31]:
pipe_gpt = pipeline("text-generation", model=model_gpt, tokenizer=tokenizer_gpt)
prompt_gpt = "It's prequel time again. In December, we'll get to "
generated_gpt = pipe_gpt(prompt_gpt, streamer=TextStreamer(tokenizer_gpt), max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It's prequel time again. In December, we'll get to  get more info on the upcoming sequel, and with the new trailer set to be revealed soon, we'll know about the upcoming trailer for the upcoming game. Meanwhile, let me know if you like some of the interesting characters and plot moments from the game. 
A new trailer from the game
I've had a chance to talk to a few of the characters through the game, and they all tell me they really thought that they would have an opportunity to fight back against the Dark Lords of Draxheim. That is why this news was sent from a different perspective to them from the Dark Lord of Draxheim, to be


In [17]:
text_mistral = generated[0]['generated_text']
text_gpt = generated_gpt[0]['generated_text']

In [43]:
text_mistral, text_gpt

('It\'s prequel time again. In December, we\'ll get to 1984\'s "The Terminator," and in January, we\'ll get to 1989\'s "Terminator 2: Judgment Day."\n\nThe first film, which was directed by James Cameron, is about a cyborg assassin from the future who is sent back in time to kill Sarah Connor (Linda Hamilton), the mother of the future leader of the human resistance against the machines. The second film, which was also directed by Cameron, is about the same cyborg assassin, who is sent back in time to protect Sarah\'s son, John Connor (',
 'It\'s prequel time again. In December, we\'ll get to \xa0see one of the other games that\'s not in the "prequel" canon. In this one,\xa0 HEX \xa0is now a side quest that adds a new story line to the series. So what about Episodes 9-11? Well, if you\'ve been following the franchise for any length of time, you\'ve probably already begun following them. I think the series should take you through some of those quests if you want to go deeper into the "pr

In [22]:
# Define a prompt
prompt = "Your text prompt here."

# Encode the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt') # Send to GPU (device 0)

# Generate text with both models
output_ids_mistral = model.generate(input_ids, max_length=50)  # Adjust max_length as needed
output_ids_gpt = model_gpt.generate(input_ids, max_length=50)  # Adjust max_length as needed

# Decode the generated ids to text
generated_text_mistral = tokenizer.decode(output_ids_mistral[0], skip_special_tokens=True)
generated_text_gpt = tokenizer_gpt.decode(output_ids_gpt[0], skip_special_tokens=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/lustre/hpc/pheno/inar/mambaforge/envs/torch39/lib/python3.9/site-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for o

In [33]:
import torch
from torch.nn import CrossEntropyLoss

def compute_loss(model, tokenizer, text):
    # Tokenize and prepare input and labels
    device = model.device
    inputs = tokenizer.encode(text, return_tensors="pt")
    inputs = inputs.to(device)
    labels = inputs.clone()

    # Get model predictions
    with torch.no_grad():
        outputs = model(inputs, labels=labels)
        logits = outputs.logits

    # Shift logits and labels to align for loss calculation
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()

    # Calculate loss
    loss_fct = CrossEntropyLoss()
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

    return loss.item()



In [47]:
# Compute loss for each generated text
loss_mistral_text = compute_loss(model_gpt, tokenizer_gpt, text_mistral)  
loss_gpt_text = compute_loss(model_gpt, tokenizer_gpt, text_gpt)  

print(f"GPT Loss for Mistral generated text: {loss_mistral_text}")
print(f"GPT Loss for GPT generated text: {loss_gpt_text}")


loss_mistral_mistral = compute_loss(model, tokenizer, text_mistral)  
loss_gpt_mistral = compute_loss(model, tokenizer, text_gpt)  

print(f"Mistral Loss for Mistral generated text: {loss_mistral_mistral}")



GPT Loss for Mistral generated text: 2.1475560665130615
GPT Loss for GPT generated text: 2.790302276611328
Mistral Loss for Mistral generated text: 0.901909351348877


In [48]:
# Compute loss for each generated text
loss_mistral_text = compute_loss(model_gpt, tokenizer_gpt, generated_text_mistral)  
loss_gpt_text = compute_loss(model_gpt, tokenizer_gpt, generated_text_gpt)  

print(f"Loss for Mistral generated text: {loss_mistral_text}")
print(f"Loss for GPT generated text: {loss_gpt_text}\n")

loss_mistral_mistral = compute_loss(model, tokenizer, generated_text_mistral)
loss_gpt_mistral = compute_loss(model, tokenizer, generated_text_gpt)

print(f"Mistral Loss for Mistral generated text: {loss_mistral_mistral}")
print(f"Mistral Loss for GPT generated text: {loss_gpt_mistral}")


Loss for Mistral generated text: 2.46722149848938
Loss for GPT generated text: 3.3510661125183105

Mistral Loss for Mistral generated text: 1.3705629110336304
Mistral Loss for GPT generated text: 4.150167942047119
